### Simple motion play of `Common Rig H` 

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig_h.xml`

In [2]:
xml_path = '../asset/myohuman_v0.1(mj236).mjb'
env = MuJoCoParserClass(name='myosuite',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[83]
n_geom:[82]
geom_names:['floor', 'r_pelvis', 'l_pelvis', 'sacrum', 'r_femur', 'r_tibia', 'r_fibula', 'r_talus', 'r_foot', 'r_bofoot', 'r_patella', 'l_femur', 'l_tibia', 'l_fibula', 'l_talus', 'l_foot', 'l_bofoot', 'l_patella', 'hat_spine', 'hat_jaw', 'hat_skull', 'hat_ribs_scap', 'humerus_r', 'ulna', 'radius', 'lunate', 'scaphoid', 'pisiform', 'triquetrum', 'capitate', 'trapezium', '1mc', 'thumbprox', 'thumbdist', 'trapezoid', 'hamate', '2mc', '2proxph', '2midph', '2distph', '3mc', '3proxph', '3midph', '3distph', '4mc', '4proxph', '4midph', '4distph', '5mc', '5proxph', '5midph', '5distph', 'humerus_lv', 'ulna_l', 'radius_l', 'lunate_l', 'scaphoid_l_geom_1', 'pisiform_l_geom_1', 'triquetrum_l_geom_1', 'capitate_l_geom_1', 'trapezium_l_geom_1', 'firstmc_l_geom_1', 'proximal_thumb_l_geom_1', 'distal_thumb_l_geom_1', 'trapezoid_l_geom_1', 'hamate_l_geom_1', 'secondmc_l_geom_1', '2proxph_l_geom_1', '2midph_l_geom_1', '2distph_l_geom_1', 'thirdmc_l_geom_

In [3]:
# Set which joints to control
ctrl_joint_names = env.joint_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
n_ctrl_joint = len(ctrl_joint_names)

# import pandas as pd
# pkl_path = '../data/F01A0V1.pkl'
# pd.read_pickle(pkl_path)

import pickle
pkl_path = '../data/M02F4V1.pkl'
with open(pkl_path,'rb') as f:
    data = pickle.load(f)

print(data['length'])
print(data['root'].shape)
print(data['qpos'].shape)

1136
(1136, 3)
(1136, 83)


### Load motion and playback

In [11]:
L = data['length']

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig H',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()
tick = 0
while env.is_viewer_alive():
    
    q = data['qpos'][tick,:]
    p_root = data['root'][tick,:]
    # tick = min(tick+1,L-1)
    if tick==(L-1): tick = 0
    else: tick = tick + 1
    env.set_p_root(root_name='body_root',p=p_root)
    env.forward(q=q,INCREASE_TICK=True)
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        # env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        # env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        # env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

KeyboardInterrupt: 

### Save Video

In [17]:
import cv2, os
video = []
tick = 0

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig H',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,1.2])
env.reset()

while tick < L:
    q = data['qpos'][tick,:]
    p_root = data['root'][tick,:]

    env.set_p_root(root_name='body_root',p=p_root)
    env.forward(q=q,INCREASE_TICK=True)

    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        # env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        # env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)
        env.render()   
        frame = env.grab_image()
        video.append(frame)

    tick = tick + 1

env.close_viewer()

shape = video[0].shape
video_folder = ("../video/myosuite")
if not os.path.isdir(video_folder): os.mkdir(video_folder)
video_out = cv2.VideoWriter(os.path.join(video_folder, "M02F4V1.mp4"), cv2.VideoWriter_fourcc('M','P','4','V'), 125, (shape[1], shape[0]), True)
for i in range(len(video)):
  video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
  video_out.write(video_frame)
video_out.release()


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


### First, get the sequence of two feet pose

In [ ]:
# p_rfoot_traj = np.zeros((L,3))
# p_lfoot_traj = np.zeros((L,3))
# for tick in range(L):
#     q = sample_qs[0,tick,:] # [35]
#     p_root = sample_p_roots[0,tick,:] # [3]
#     env.set_p_root(root_name='base',p=p_root)
#     env.forward(q=q,joint_idxs=joint_idxs_fwd,coupling=True)
#     # Append
#     p_rfoot_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
#     p_lfoot_traj[tick,:] = env.get_p_geom(geom_name='lfoot')

In [ ]:
# # Initialize MuJoCo viewer
# env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
#                 viewer_hide_menus=True)
# env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
# env.reset()
# tick = 0
# p_rfoot_centered_traj = np.zeros((L,3))
# p_lfoot_centered_traj = np.zeros((L,3))
# while env.is_viewer_alive():
    
#     # Update
#     q = sample_qs[0,tick,:] # [35]
#     p_root = sample_p_roots[0,tick,:] # [3]
#     p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
#     env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
#     env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
#     # Append translated foot traj
#     p_rfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
#     p_lfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='lfoot')
    
#     # Render
#     if env.loop_every(tick_every=1):
#         # Plot world frame
#         env.plot_T(p=np.zeros(3),R=np.eye(3,3),
#                    PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
#         env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
#                    PLOT_AXIS=False,label="tick:[%d]"%(tick))
#         # Plot foot
#         env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
#         env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
#         # Plot revolute joints with arrow
#         env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
#         env.render()
        
#     # Increase tick
#     if tick==(L-1): tick = 0
#     else: tick = tick + 1
# # Close MuJoCo viewer
# env.close_viewer()
# print ("Done.")

### Solve IK for two feet

In [ ]:
# p_trgt_rfoot = np.average(p_rfoot_centered_traj,axis=0)
# p_trgt_lfoot = np.average(p_lfoot_centered_traj,axis=0)
# R_trgt_rfoot = rpy2r(np.radians([0,0,0]))
# R_trgt_lfoot = rpy2r(np.radians([0,0,0]))
# # Initialize MuJoCo viewer
# env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
#                 viewer_hide_menus=True)
# env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
# env.reset()
# while env.is_viewer_alive():
#     # Update
#     q = sample_qs[0,tick,:] # [35]
#     p_root = sample_p_roots[0,tick,:] # [3]
#     p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
#     env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
#     env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
#     # Solve IK
#     ik_geom_names = ['rfoot','lfoot']
#     ik_p_trgts = [p_trgt_rfoot,p_trgt_lfoot]
#     ik_R_trgts = [R_trgt_rfoot,R_trgt_lfoot]
#     err_traj = np.zeros(30)
#     for ik_tick in range(30):
#         J_list,ik_err_list = [],[]
#         for ik_idx,ik_geom_name in enumerate(ik_geom_names):
#             ik_p_trgt = ik_p_trgts[ik_idx]
#             ik_R_trgt = ik_R_trgts[ik_idx]
#             J,ik_err = env.get_ik_ingredients_geom(
#                 geom_name=ik_geom_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
#                 IK_P=True,IK_R=True)
#             J_list.append(J)
#             ik_err_list.append(ik_err)
#         J_stack      = np.vstack(J_list)
#         ik_err_stack = np.hstack(ik_err_list)
#         err_traj[ik_tick] = np.max(np.abs(ik_err_stack))
#         dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
#         q = q + dq[joint_idxs_jac]
#         env.forward(q=q,joint_idxs=joint_idxs_fwd,coupling=True)
        
#     # Render
#     if env.loop_every(tick_every=1):
#         # Plot world frame
#         env.plot_T(p=np.zeros(3),R=np.eye(3,3),
#                    PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
#         env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
#                    PLOT_AXIS=False,label="tick:[%d]"%(tick))
#         # Plot foot
#         env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
#         env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
#         # Plot revolute joints with arrow
#         env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
#         env.render()
        
#     # Increase tick
#     if tick==(L-1): tick = 0
#     else: tick = tick + 1
# # Close MuJoCo viewer
# env.close_viewer()
# print ("Done.")